In [1]:
import os
import string
import requests
import glob
import pandas as pd 
from bs4 import BeautifulSoup
from PIL import Image

In [2]:
path = "lexile/raw/book-reviews/"
files = [f for f in glob.glob(path + "*.html", recursive=True)]
files[:10]

['lexile/raw/book-reviews/the-third-twin.html',
 'lexile/raw/book-reviews/small-damages.html',
 'lexile/raw/book-reviews/the-school-for-good-and-evil-book-1.html',
 'lexile/raw/book-reviews/agent-of-chaos-the-x-files-origins-book-1.html',
 'lexile/raw/book-reviews/crossing-ebenezer-creek.html',
 'lexile/raw/book-reviews/the-real-mccoys-book-1.html',
 'lexile/raw/book-reviews/miss-educated-an-upper-class-novel-2.html',
 'lexile/raw/book-reviews/flora-and-the-flamingo.html',
 'lexile/raw/book-reviews/big-snow.html',
 'lexile/raw/book-reviews/english-roses-too-good-to-be-true.html']

In [3]:
file_names = []

for file in files:
    f = file.split("/")[3]
    file_names.append(f[:-5])

In [4]:
csm_ages = []
cover_src = []

for file in files:
    with open(file, 'r') as f: 
        soup = BeautifulSoup(f.read(), 'html.parser')
        csm_ages.append(soup.find(class_="csm-green-age").get_text().replace("age ", "")[:-1])
        cover = soup.find(class_="field field-name-field-product-image field-type-image field-label-hidden")
        cover_src.append(cover.find("img").get('src'))

In [5]:
filler = ["_" for _ in range(len(csm_ages))]

In [6]:
names = [x+y+z for x,y,z in zip(csm_ages,filler,file_names)]

In [7]:
info = dict(zip(names, cover_src))

In [8]:
!mkdir 'covers'

mkdir: cannot create directory ‘covers’: File exists


In [9]:
for file in files:
    with open(file, 'r') as f: 
        soup = BeautifulSoup(f.read(), 'html.parser')
        for k, v in info.items():
            if ".jpeg?" in v:
                if not os.path.isfile("./covers/" + k + '.jpeg'):
                    with open('./covers/' + k + '.jpeg', 'wb') as f:
                            f.write(requests.get(v).content)
            elif '.jpg?' in v:
                if not os.path.isfile("./covers/" + k + '.jpg'):
                    with open('./covers/' + k + '.jpg', 'wb') as f:
                        f.write(requests.get(v).content)
            elif '.png?' in v:
                if not os.path.isfile("./covers/" + k + '.png'):
                    with open('./covers/' + k + '.png', 'wb') as f:
                            f.write(requests.get(v).content)   

In [10]:
!mkdir covers-jpg

mkdir: cannot create directory ‘covers-jpg’: File exists


In [11]:
path2 = "covers/*"
path3 = "covers-jpg/"
files = [f for f in glob.glob(path2, recursive=True)]

In [14]:
def convert (file):
    im = Image.open(file)
    rgb_im = im.convert('RGB')
    new_file = rgb_im.save(path3+file[7:].split(".")[0]+".jpg")
    return new_file

In [17]:
error = []

for file in files:
    try:
        convert(file)
    except OSError:
        error.append(file)

['covers/13_the-100-book-1.jpg', 'covers/4_the-new-adventures-of-curious-george.jpg', 'covers/8_vince-carter-choose-your-course.jpg', 'covers/13_the-falconer-book-1.jpg', 'covers/14_layoverland.jpg', 'covers/2_tickle-time.jpg']


In [18]:
error

['covers/13_the-100-book-1.jpg',
 'covers/4_the-new-adventures-of-curious-george.jpg',
 'covers/8_vince-carter-choose-your-course.jpg',
 'covers/13_the-falconer-book-1.jpg',
 'covers/14_layoverland.jpg',
 'covers/2_tickle-time.jpg']